In [1]:
import pandas as pd
import numpy as np

In [2]:
from pyport.portopt import opt, objfunc

In [3]:
from pandas_datareader import data

In [4]:
"""Define parameters for data request."""

tickers = ['AGG', 'EBND', 'URTH', 'EEM']
data_source = 'yahoo'
start_date = '2014-03-31'
end_date = '2018-03-31'

In [5]:
"""Request data, ensure sorted by ascending date, and normalise."""

panel_data = data.DataReader(tickers, data_source, start_date, end_date)
px_data = panel_data.loc['Adj Close'].dropna().sort_index(ascending=True)
px_data = px_data / px_data.iloc[0]

In [6]:
"""
TEST CASE
Add constraints: Portfolio weights must sum to 100%,
Add boundaries: Long only, no short positions.
"""

cons = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
]

bnds = [(0, 1) for x in px_data.columns]

In [7]:
"""
Define the relaxation tolerance allowed for init_func.
For example, if prim_func is max_er and init_func is max_dr,
relax_tolderance=0.1 and num_of_steps=10 results in a tolerance of 10% from max_dr optimal in favour of max_er,
performed in 10 steps.
"""

relax_tolerance = 0.1
num_of_steps = 10

In [8]:
"""Run optimisation."""

opt_results = opt.dual_target_optimisation(
    objfunc.max_er, objfunc.min_vol, px_data, 
    relax_tol=relax_tolerance, steps=num_of_steps,
    rf=0.0, scaling_fact=252, 
    constraints=cons, bounds=bnds)

Total time: 3.166881799697876 secs


In [9]:
"""View results."""

opt_results

,max_er,min_vol,AGG,EBND,EEM,URTH
step,,,,,,
0,0.026816,0.028786,0.9171,0.0,0.0,0.0829
1,0.028341,0.029106,0.8875,0.0,0.0,0.1125
2,0.028981,0.029425,0.8750,0.0,0.0,0.1250
3,0.029476,0.029745,0.8654,0.0,0.0,0.1346
4,0.029898,0.030065,0.8573,0.0,0.0,0.1427
5,0.030272,0.030385,0.8500,0.0,0.0,0.1500
6,0.030613,0.030705,0.8434,0.0,0.0,0.1566
7,0.030929,0.031025,0.8373,0.0,0.0,0.1627
8,0.031226,0.031345,0.8315,0.0,0.0,0.1685
